In [1]:
from os import listdir
from os.path import isdir, join
import librosa
import random
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features
import scipy

In [2]:
# Dataset path and view possible targets
dataset_path = './recording9'
for name in listdir(dataset_path):
    if isdir(join(dataset_path, name)):
        print(name)

backgroundNoise
ㄏㄧㄡ
ㄟ
他
你
其他
吼
啦
嗯
好
我
的
著
那
阿


In [3]:
# Create an all targets list
all_targets = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_targets)

['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '他', '你', '其他', '吼', '啦', '嗯', '好', '我', '的', '著', '那', '阿']


In [4]:
# See how many files are in each
num_samples = 0
for target in all_targets:
    print(len(listdir(join(dataset_path, target))))
    num_samples += len(listdir(join(dataset_path, target)))
print('Total samples:', num_samples)

500
1240
1240
1240
1240
9640
2451
1240
1240
1301
1240
1333
1301
1828
1240
Total samples: 28274


In [5]:
# Settings
target_list = all_targets
#feature_sets_file = 'recording1.npz'
perc_keep_samples = 1 #取整個資料集當中的100%
val_ratio = 0.2 #0.2
test_ratio = 0.2 #0.2
sample_rate = 16000 #取樣率

In [6]:
# Create list of filenames along with ground truth vector (y)
filenames = []
y = []
for index, target in enumerate(target_list):
    print(join(dataset_path, target))
    filenames.append(listdir(join(dataset_path, target)))
    y.append(np.ones(len(filenames[index])) * index)

./recording9\backgroundNoise
./recording9\ㄏㄧㄡ
./recording9\ㄟ
./recording9\他
./recording9\你
./recording9\其他
./recording9\吼
./recording9\啦
./recording9\嗯
./recording9\好
./recording9\我
./recording9\的
./recording9\著
./recording9\那
./recording9\阿


In [7]:
# Check ground truth Y vector
print(y)
for item in y:
    print(len(item))

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0

In [13]:
# Flatten filename and y vectors(轉成1D張量)
filenames = [item for sublist in filenames for item in sublist]
y = [item for sublist in y for item in sublist]

In [14]:
# Associate filenames with true output and shuffle
filenames_y = list(zip(filenames, y))#filename跟y壓縮在 起
random.shuffle(filenames_y)#打散
filenames, y = zip(*filenames_y)#解壓縮

In [15]:
# Only keep the specified number of samples (shorter extraction/training)
#只用整個資料集的100%
print(len(filenames))
filenames = filenames[:int(len(filenames) * perc_keep_samples)]
print(len(filenames))

28274
28274


In [16]:
# Calculate validation and test set sizes(驗證集、測試集使用整個資料集10%當中的各10%)
val_set_size = int(len(filenames) * val_ratio)
test_set_size = int(len(filenames) * test_ratio)
print(val_set_size)
print(test_set_size)

5654
5654


In [17]:
# Break dataset apart into train, validation, and test sets(分割資料集，filename部分)
filenames_val = filenames[:val_set_size]
filenames_test = filenames[val_set_size:(val_set_size + test_set_size)]
filenames_train = filenames[(val_set_size + test_set_size):]

In [18]:
# Break y apart into train, validation, and test sets(分割資料集，target部分)
y_orig_val = y[:val_set_size]
y_orig_test = y[val_set_size:(val_set_size + test_set_size)]
y_orig_train = y[(val_set_size + test_set_size):]

In [20]:
print(filenames)

('noise_shifting-time_changing-pitch_我21-1.wav', '60_1_3_noise_augment.wav', 'noise_shifting-time_changing-pitch_ㄟ19-3.wav', 'shifting_time_啦15-1.wav', 'noise_shifting-time_changing-pitch_啦32-2.wav', '78_2_5_noise_augment.wav', '35_3_2_shifting_time.wav', 'noise_augment_那9-2.wav', '15_2_2_shifting_time.wav', 'time_masking_阿25-1.wav', '那5_2_2_shifting_time.wav', '吼7_2_2_noise_augment.wav', 'time_masking_吼7-4.wav', '23_1_1_shifting_time.wav', 'frequency_masking_我16-5.wav', '103_2_5_shifting_time.wav', 'noise_augment_的10-4.wav', '98_2_3_shifting_time.wav', 'frequency_masking_那18-5.wav', '18_2_4_shifting_time.wav', 'time_masking_ㄏㄧㄡ5-5.wav', '1_1_3_noise_shifting-time_changing-pitch.wav', 'noise_shifting-time_changing-pitch_吼23-2.wav', '吼14_3_3_changing_pitch.wav', 'backgroundNoise327.wav', '88_3_3_noise_shifting-time_changing-pitch.wav', '101_2_4_noise_shifting-time_changing-pitch.wav', '69_2_1_noise_shifting-time_changing-pitch.wav', 'changing_pitch_ㄏㄧㄡ32-3.wav', 'time_masking_我1-2.wav',

In [31]:

# Function: Create MFCC from given path
def calc_fbank(path):
    
    # Load wavefile(取得時序、取樣率)
    signal, fs = librosa.load(path, sr=sample_rate)
    signal = signal.astype(np.float64)
    # normalize data
    signal = (signal - signal.mean()) / (signal.max() - signal.min())

    # Create MFCCs from sound clip
    fbank = python_speech_features.base.logfbank(signal,
                                                 samplerate=sample_rate,
                                                 winlen=0.025,
                                                 winstep=0.01,
                                                 nfilt=26,
                                                 nfft=512,
                                                 lowfreq=0,
                                                 highfreq=None,
                                                 preemph=0.97)
    return fbank

In [32]:
print(len(filenames_train))

16966


In [33]:
# Function: Create MFCCs, keeping only ones of desired length
#提取完好的音檔
def extract_features(in_files, in_y):
    prob_cnt = 0
    out_x = []
    out_y = []
        
    for index, filename in enumerate(in_files):
    
        # Create path from given filename and target item
        path = join(dataset_path, target_list[int(in_y[index])], 
                    filename)
        
        # Check to make sure we're reading a .wav file
        if not path.endswith('.wav'):
            continue

        # Create MFCCs
        fbank = calc_fbank(path)
            
        # Only keep MFCCs with given length
        if fbank.shape[0] == 49:
            out_x.append(fbank)
            out_y.append(in_y[index])
        else:
            print(filename)
            print('Dropped:', index, fbank.shape)
            prob_cnt += 1

    return out_x, out_y, prob_cnt

In [34]:
# Create train, validation, and test sets
total = 0

x_train, y_train, prob = extract_features(filenames_train, 
                                          y_orig_train)
total += prob

x_val, y_val, prob = extract_features(filenames_val, y_orig_val)
total += prob

x_test, y_test, prob = extract_features(filenames_test, y_orig_test)
total += prob

print("total:" + str(total))

2_2_1_changing_pitch.wav
Dropped: 1196 (50, 26)
2_2_4_noise_augment.wav
Dropped: 3342 (50, 26)
2_2_5_noise_augment.wav
Dropped: 4602 (50, 26)
2_2_3_noise_augment.wav
Dropped: 5331 (50, 26)
2_2_1_shifting_time.wav
Dropped: 6799 (50, 26)
2_2_4_noise_shifting-time_changing-pitch.wav
Dropped: 8265 (50, 26)
2_2_2_noise_shifting-time_changing-pitch.wav
Dropped: 8478 (50, 26)
2_2_2_changing_pitch.wav
Dropped: 11202 (50, 26)
2_2_4_shifting_time.wav
Dropped: 11602 (50, 26)
2_2.wav
Dropped: 12387 (50, 26)
2_2_5_shifting_time.wav
Dropped: 13195 (50, 26)
2_2_5_noise_shifting-time_changing-pitch.wav
Dropped: 13200 (50, 26)
2_2_2_shifting_time.wav
Dropped: 13875 (50, 26)
2_2_1_noise_augment.wav
Dropped: 14930 (50, 26)
2_2_3_changing_pitch.wav
Dropped: 16885 (50, 26)
2_2_3_noise_shifting-time_changing-pitch.wav
Dropped: 121 (50, 26)
2_2_2_noise_augment.wav
Dropped: 1124 (50, 26)
2_2_1_noise_shifting-time_changing-pitch.wav
Dropped: 1849 (50, 26)
2_2_5_changing_pitch.wav
Dropped: 2245 (50, 26)
2_2_3_s

In [35]:
#儲存資料
np.savez('./npz/recording9_fbank.npz', 
         x_train=x_train, 
         y_train=y_train, 
         x_val=x_val, 
         y_val=y_val, 
         x_test=x_test, 
         y_test=y_test)

In [ ]:
feature_sets = np.load('./npz/recording9_fbank.npz')
feature_sets.files

In [ ]:
print(len(feature_sets['x_train']))
print(len(feature_sets['x_test']))
print(len(feature_sets['x_val']))